## 1. Приведение разметки к формату BIO

In [ ]:
import pandas as pd

In [ ]:
!wget https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/train_aspects.txt
!wget https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/train_reviews.txt

--2022-12-29 00:34:57--  https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/train_aspects.txt
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_aspects.txt [following]
--2022-12-29 00:34:57--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_aspects.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228391 (223K) [text/plain]
Saving to: ‘train_aspects.txt.2’

train_aspects.txt.2 100%[===================>] 223.04K  --.-KB/s    in 0.02s   

2022-12-29 00:34:57 (12.3 MB/s) - ‘train_aspects.txt

In [ ]:
train_aspects = pd.read_csv('train_aspects.txt', delimiter='\t', names=['text_id', 'category', 'mention', 'start', 'end', 'sentiment'])

In [ ]:
train_aspects.head()

,text_id,category,mention,start,end,sentiment
0,3976,Whole,ресторане,71,80,neutral
1,3976,Whole,ресторанах,198,208,neutral
2,3976,Whole,ресторане,256,265,neutral
3,3976,Service,Столик бронировали,267,285,neutral
4,3976,Service,администратор,322,335,positive


In [ ]:
train_aspects.shape

(4763, 6)

In [ ]:
mentions_info = []
for row in train_aspects.itertuples():
    if len(row.mention.split()) == 1:
         mentions_info.append({'text_id': row.text_id, 'mention': row.mention, 'category': 'B-' + row.category, 'sentiment': row.sentiment})
    else:
        mentions_info.append({'text_id': row.text_id, 'mention': row.mention.split()[0], 'category': 'B-' + row.category, 'sentiment': row.sentiment})
        for i in range(1, len(row.mention.split())):
            mentions_info.append({'text_id': row.text_id, 'mention': row.mention.split()[i], 'category': 'I-' + row.category, 'sentiment': row.sentiment})

In [ ]:
len(mentions_info)

6229

In [ ]:
mentions_info[:10]

[{'text_id': 3976,
  'mention': 'ресторане',
  'category': 'B-Whole',
  'sentiment': 'neutral'},
 {'text_id': 3976,
  'mention': 'ресторанах',
  'category': 'B-Whole',
  'sentiment': 'neutral'},
 {'text_id': 3976,
  'mention': 'ресторане',
  'category': 'B-Whole',
  'sentiment': 'neutral'},
 {'text_id': 3976,
  'mention': 'Столик',
  'category': 'B-Service',
  'sentiment': 'neutral'},
 {'text_id': 3976,
  'mention': 'бронировали',
  'category': 'I-Service',
  'sentiment': 'neutral'},
 {'text_id': 3976,
  'mention': 'администратор',
  'category': 'B-Service',
  'sentiment': 'positive'},
 {'text_id': 3976,
  'mention': 'предварительный',
  'category': 'B-Service',
  'sentiment': 'positive'},
 {'text_id': 3976,
  'mention': 'заказ',
  'category': 'I-Service',
  'sentiment': 'positive'},
 {'text_id': 3976,
  'mention': 'ресторан',
  'category': 'B-Whole',
  'sentiment': 'neutral'},
 {'text_id': 3976,
  'mention': 'ресторане',
  'category': 'B-Whole',
  'sentiment': 'neutral'}]

In [ ]:
train_texts = pd.read_csv('train_reviews.txt', delimiter='\t', names=['text_id','text'])

In [ ]:
train_texts.head()

,text_id,text
0,3976,"День 8-го марта прошёл, можно и итоги подвести..."
1,30808,Отмечали в этом ресторане день рождение на пер...
2,14031,Хочу поделиться своим впечатлением от посещени...
3,2495,Добрый день! Были вчера с друзьями в этом кафе...
4,38835,Отметили с мужем годовщину свадьбы 6 ноября в ...


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
sentences_info = []
for row_text in train_texts.itertuples():
    for sentence in sent_tokenize(row_text.text):
      s = []
      for token in word_tokenize(sentence):
        if token == "``" or token == "''":
          token = '"'
        s.append([row_text.text_id, 'O', token, row_text.text.index(token), row_text.text.index(token) + len(token), None])
      sentences_info.append(s)

In [ ]:
for mention in mentions_info:
    for sentence in sentences_info:
      for token in sentence:
        if mention['text_id'] == token[0] and mention['mention'] == token[2]:
            token[1] = mention['category']
            token[5] = mention['sentiment']

## 2. Выделение признаков и обучение модели

In [ ]:
import copy

In [ ]:
sentences_info_for_train = copy.deepcopy(sentences_info)

In [ ]:
for sentence in sentences_info_for_train:
    for token in sentence:
      token.pop(0)
      token.pop(2)
      token.pop(2)
      token.pop(2)

In [ ]:
print(len(sentences_info_for_train))
print(len(sentences_info_for_train) * 0.8)
print(len(sentences_info_for_train) * 0.2)

2976
2380.8
595.2


In [ ]:
train_sentences = sentences_info_for_train[:2381]
test_sentences = sentences_info_for_train[2381:2977]

In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
def word2features(sentence, index):
    word = sentence[index][1]
    features = {'word.lower()': word.lower(), 'word.isupper()': word.isupper(), 'pos': str(morph.parse(word)[0].tag.POS), 'BOS': False}
    if index > 0:
      pass
    else:
        features['BOS'] = True
    if index < len(sentence) - 1:
      pass
    else:
        features['EOS'] = True
    return features

def sent2features(sentence):
    return [word2features(sentence, index) for index in range(len(sentence))]

def sent2labels(sentence):
    return [label for label, token in sentence]

In [ ]:
!pip install -U 'scikit-learn<0.24'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sklearn_crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [ ]:
X_train = [sent2features(s) for s in train_sentences]
y_train = [sent2labels(s) for s in train_sentences]

X_test = [sent2features(s) for s in test_sentences]
y_test = [sent2labels(s) for s in test_sentences]

In [ ]:
crf = sklearn_crfsuite.CRF(algorithm='ap')
crf.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='ap', keep_tempfiles=None)

In [ ]:
labels = list(crf.classes_)

In [ ]:
y_pred = crf.predict(X_test)
metrics.flat_accuracy_score(y_test, y_pred)

0.9010305070911131

In [ ]:
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pickle.dump(crf, open('/content/drive/My Drive/crf.sav', 'wb'))

In [ ]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=['O', 'B-Food', 'I-Food', 'B-Interior', 'I-Interior', 'B-Price', 'I-Price', 'B-Service', 'I-Service', 'B-Whole', 'I-Whole'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


              precision    recall  f1-score   support

           O      0.921     0.979     0.949      8232
      B-Food      0.660     0.658     0.659       404
      I-Food      0.736     0.338     0.463       314
  B-Interior      0.874     0.539     0.667       180
  I-Interior      0.435     0.196     0.270        51
     B-Price      0.760     0.655     0.704        29
     I-Price      0.333     0.167     0.222         6
   B-Service      0.893     0.486     0.629       311
   I-Service      0.529     0.261     0.350        69
     B-Whole      0.739     0.614     0.671       166
     I-Whole      1.000     0.128     0.227        39

    accuracy                          0.901      9801
   macro avg      0.716     0.456     0.528      9801
weighted avg      0.894     0.901     0.890      9801

